# 12_join_table_accre

## Environment

In [2]:
# import packages
import pandas as pd
import numpy as np
import os
#import janitor
from glob import glob
import time

## Set Directories

In [11]:
data_path = '/data/p_dsi/teams2022/bridgestone_data/data/'
sales_file_list = glob(data_path + 'sales_2*.csv')
local_path = '/home/conawws1/repeat_customers/'

In [12]:
if not os.path.exists(local_path + "data/merged_data"):
    os.mkdir(local_path + "data/merged_data")

## Data Read Functions

In [20]:
def read_sales(file):
    col_list = ['STORE_ID','TRAN_ID','DATE','ARTICLE_ID','INDIV_ID','VEHICLE_ID','UNITS','SALES']
    df = pd.read_csv(file
                  ,sep='|'
                  ,usecols=col_list
                  #,parse_dates=['DATE']
                  #,date_parser=date_parser
                  ,dtype = {'STORE_ID':'category'
                            ,'TRAN_ID':np.int32
                            ,'DATE':'category'
                            ,'ARTICLE_ID':np.int32
                            ,'INDIV_ID':np.float16 # int32 throws error claiming float value
                            ,'VEHICLE_ID':np.int32
                            ,'UNITS':np.int8
                            ,'SALES':np.float16
                            }
                 )
    df.columns = ['store_id', 'tran_id', 'date', 'article_id','indiv_id','vehicle_id', 'units', 'sales']
    
    return df

In [45]:
def read_individuals(file):
    
    col_list = ['MZB_INDIV_ID','EMAIL_OPTIN_IND','AH1_RES_BUS_INDC','SUPP1_BUS_PANDER']
    
    individuals = pd.read_csv(file
                            ,sep=','
                            ,usecols=col_list
                            ,dtype = {'MZB_INDIV_ID':np.float16
                                        ,'EMAIL_OPTIN_IND':'category'
                                        ,'AH1_RES_BUS_INDC':'category'
                                        ,'SUPP1_BUS_PANDER':'category'
                                     } 
                            )
    
    individuals.columns = ['indiv_id', 'email_optin_ind', 'ah1_res_bus_indc','supp1_bus_pander']

    individuals = individuals[(individuals['ah1_res_bus_indc'] == 'R') & (individuals['supp1_bus_pander'] == 'N') & (individuals['email_optin_ind'] == 'Y')]
    individuals.drop(['ah1_res_bus_indc', 'supp1_bus_pander', 'email_optin_ind'], axis=1, inplace=True)

    return individuals

In [22]:
def read_products(file):
    col_list = ['ARTICLE_ID', 'PROD_GROUP_CODE', 'PROD_GROUP_DESC', 'CATEGORY_CODE',
            'CATEGORY_DESC', 'SEGMENT_CODE', 'SEGMENT_DESC', 'CLASS_CODE',
            'CLASS_DESC', 'DISCOUNT_FLAG', 'CROSS_SECTION', 'ASPECT_RATIO',
            'RIM_SIZE']
    products = pd.read_csv(file
                            ,sep='|'
                            ,usecols=col_list
                            ,dtype = {'ARTICLE_ID':np.int32, 'PROD_GROUP_CODE':'category'
                                    , 'PROD_GROUP_DESC':'category', 'CATEGORY_CODE':'category'
                                    ,'CATEGORY_DESC':'category', 'SEGMENT_CODE':'category'
                                    , 'SEGMENT_DESC':'category', 'CLASS_CODE':'category'
                                    , 'CLASS_DESC':'category', 'DISCOUNT_FLAG':'category'
                                    , 'CROSS_SECTION':'category', 'ASPECT_RATIO':'category',
                                    'RIM_SIZE':'category'
                                     }
                            )

    products.columns = ['article_id', 'prod_group_code', 'prod_group_desc', 'category_code',
                       'category_desc', 'segment_code', 'segment_desc', 'class_code',
                       'class_desc', 'discount_flag', 'cross_section', 'aspect_ratio',
                       'rim_size']
    
    return products

In [23]:
def read_stores(file):
    col_list = ['STORE_ID','STATE_CODE','ZIP_CODE','MSA']
    stores = pd.read_csv(file
                        ,sep='|'
                        ,usecols=col_list
                        ,dtype = {'STORE_ID':'category'
                                    ,'STATE_CODE':'category'
                                    ,'ZIP_CODE':'category'
                                    ,'MSA':'category'
                                 }
                        )

    stores.columns = ['store_id', 'state_code', 'zip_code', 'msa']
    
    return stores

In [24]:
def read_vehicles(file):
    col_list = ['VEHICLE_ID','MAKE','MODEL','SUB_MODEL','MODEL_YEAR']
    vehicles = pd.read_csv(file
                            ,sep='|'
                            ,usecols=col_list
                            ,dtype = {'VEHICLE_ID':np.int32
                                    ,'MAKE':'category'
                                    ,'MODEL':'category'
                                    ,'SUB_MODEL':'category'
                                    ,'MODEL_YEAR':np.int16
                                     }
                            )
    
    vehicles.columns = ['vehicle_id', 'make', 'model', 'sub_model', 'model_year']
    
    return vehicles

## Process Data

In [47]:
individual = read_individuals(data_path + '/individual.csv')

In [25]:
# read non-sales tables
individual = read_individuals(data_path + '/individual.csv')
product = read_products(data_path + '/product.csv')
store = read_stores(data_path + '/store.csv')
vehicle = read_vehicles(data_path + '/vehicle.csv')

### One Sales File and Merge Test

In [32]:
# read sales file for one month
file = sales_file_list[0]
sale = read_sales(file)

Index(['STORE_ID', 'TRAN_ID', 'DATE', 'ARTICLE_ID', 'INDIV_ID', 'VEHICLE_ID',
       'UNITS', 'SALES'],
      dtype='object')
2.04 min


In [51]:
# joined tables for each sales month

new_file_list = []

# merge ancillary tables
mega_table = sale.merge(product, on = 'article_id', how = 'inner').\
    merge(store, on = 'store_id', how = 'inner').\
    merge(individual, on = 'indiv_id', how = 'inner').\
    merge(vehicle, on = 'vehicle_id', how = 'inner')
col_list = list(mega_table.columns)

# date fields
date = file[-12:-4]
mega_table["year"] = date[:4]
mega_table['month'] = date[4:-2]

# new file name
new_filename = date + '.csv'
new_file_list.append(new_filename)


# write out merged file for one month

mega_table.to_csv(local_path + "data/merged_data/" + new_filename)

0.54 min


In [54]:
mega_table

,store_id,tran_id,date,article_id,units,sales,indiv_id,prod_group_code,prod_group_desc,category_code,...,aspect_ratio,rim_size,state_code,zip_code,msa,vehicle_id,make,model,sub_model,model_year


---
---
---
---

### Sales Files and Merge Loop (after test)

In [ ]:
# joined tables for each sales month

new_file_list = []

for file in sales_file_list:
    
    # read sales file for one month
    sale = read_sales(data_path + "/" + file)
    
    # merge ancillary tables
    mega_table = sale.merge(product, on = 'article_id', how = 'inner').\
        merge(store, on = 'store_id', how = 'inner').\
        merge(individual, on = 'indiv_id', how = 'inner').\
        merge(vehicle, on = 'vehicle_id', how = 'inner')
    col_list = list(mega_table.columns)
    
    # define new file name and date fields
    new_filename = file[6:]
    new_file_list.append(new_filename)
    mega_table["year"] = new_filename[:4]
    mega_table['month'] = new_filename[4:-4]
    
    # write out merged file for one month
    mega_table.to_parquet(local_path + "data/merged_data/" + new_filename)


In [ ]:
# all the data

df = pd.DataFrame(columns = col_list)
for file in new_file_list: 
    if os.path.isfile(local_path + "/data/merged_data/" + file + ".csv"):
        df1 = pd.read_parquet(local_path + "/data/merged_data/" + file + ".csv")
        df = pd.concat([df1, df], axis = 0)
        df = df.reset_index(drop = True)

In [ ]:
df.to_parquet(local_path + "data/merged_data/total_dataset.parquet", index = False)